In [1]:
# Importing Pandas, Numpy, and datetime 

import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# Importing the SQL Alchemy tools 

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [3]:
# importing Matplotlib tools 

%matplotlib inline 
from matplotlib import style 
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

# Setting up SQL to Reflect Tables into SQLAlchemy ORM 

In [4]:
# Creating the engine 
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# Reflecting the existing database into a new model
Base = automap_base()

# reflecting the tables 
Base.prepare(engine, reflect = True)

In [6]:
# Viewing all the classes that Automap found 
Base.classes.keys()

['measurement', 'station']

In [7]:
# Saving References to each Table 
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
session = Session(engine)

# Sql Alchemy Set up complete
-----------------------------------------------------------------------------

# Constructing the Classes into Dataframes for the hawaii.sqlite data-table.

In [9]:
# Having this to have a quick look at the first rows of each table

#q = session.query(measurement).first()
#q.__dict__
#p = session.query(station).first()
#p.__dict__ 

Creating the DataFrame for the station table data

In [10]:
# Querying the station Table data for all  
station_query = engine.execute('SELECT * FROM station').fetchall()

# Converting the station_query into a Dataframe to hold the data
station_df = pd.DataFrame(station_query)

# Renaming the station_df to the columns respective names
station_df = station_df.rename(columns={1:'station', 2:'name', 3:'latitude', 4:'longitude',5:'elevation'})

# Removing the Primary Key id column from the table of data imported form SQLite
station_df = station_df.drop(0,1)

# Presenting the station_df table
station_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


Creating the DataFrame for the measurement table data

In [11]:
# Querying the measurement Table data for all  
measurement_query = engine.execute('SELECT * FROM measurement').fetchall()

# Converting the measurement_query into a Dataframe to hold the data
measurement_df = pd.DataFrame(measurement_query)

# Renaming the Measurement_df to the columns respective names
measurement_df = measurement_df.rename(columns={1: 'station', 2:'date', 3:'precipitation', 4:'tobs'})

# Removing the Primary Key id column from the table of data imported form SQLite
measurement_df = measurement_df.drop(0,1)

# Presenting the measurement_df table
measurement_df.tail(5)

,station,date,precipitation,tobs
19545,USC00516128,2017-08-19,0.09,71.0
19546,USC00516128,2017-08-20,NaN,78.0
19547,USC00516128,2017-08-21,0.56,76.0
19548,USC00516128,2017-08-22,0.50,76.0
19549,USC00516128,2017-08-23,0.45,76.0


Merging both tables together by comparing the station names and iterating each row to put the station data into it

In [12]:
for i in range(0,len(measurement_df)):
    
    for j in range(0,len(station_df)):
        
        if measurement_df.iloc[i,0] == station_df.iloc[j,0]:
            
            measurement_df['station_name'] = station_df.iloc[j,1]
            measurement_df['station_latitude'] = station_df.iloc[j,2]
            measurement_df['station_longitude'] = station_df.iloc[j,3]
            measurement_df['station_elevation'] = station_df.iloc[j,4]

In [13]:
measurement_station_df = measurement_df

measurement_station_df.tail()

,station,date,precipitation,tobs,station_name,station_latitude,station_longitude,station_elevation
19545,USC00516128,2017-08-19,0.09,71.0,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4
19546,USC00516128,2017-08-20,NaN,78.0,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4
19547,USC00516128,2017-08-21,0.56,76.0,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4
19548,USC00516128,2017-08-22,0.50,76.0,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4
19549,USC00516128,2017-08-23,0.45,76.0,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4


# DataFrames construction complete
# ---------------------------------------------------

# Precipitation Analysis

Configuring the Dataframe for the precipitation analysis

In [14]:
# designing a dataframe to solely hold the data and precipitation data.
solely_precipitation = measurement_station_df[['precipitation','date']]

# setting the index of solely_precipitation to to 'date' 
solely_precipitation = solely_precipitation.set_index('date')

# sorting the index by the last twelve months.
# -----------

solely_precipitation = solely_precipitation.sort_index(ascending = True)

solely_precipitation.head()

,precipitation,date
0,0.08,2010-01-01
1,0.00,2010-01-02
2,0.00,2010-01-03
3,0.00,2010-01-04
4,NaN,2010-01-06


In [ ]:
plt.hist(solely_precipitation)


In [17]:
solely_precipitation.describe()

,precipitation
count,18103.000000
mean,0.160644
std,0.468746
min,0.000000
25%,0.000000
50%,0.010000
75%,0.110000
max,11.530000


In [18]:
solely_precipitation.max()

precipitation    11.53
dtype: float64

In [ ]:
# Flattening sets in a list 
flattened = [item for sublist in l for item in sublist]

In [ ]:
#flattened.clear()

In [ ]:
flattened